In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
val=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
sub=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv")

In [3]:
zero, one =np.bincount(df['Class'])

In [4]:
zero_df=df.loc[df['Class']==0]
one_df=df.loc[df['Class']==1]

In [5]:
zero_df=zero_df.dropna()
zero_df.shape

(446, 58)

In [6]:
one_df=one_df.fillna(df.median(numeric_only=True))
zero_df=zero_df.fillna(df.median(numeric_only=True))

In [7]:
zero_df=zero_df.drop(zero_df.sample(106).index)

In [8]:
data=pd.concat([one_df,zero_df])

In [9]:
data=data.sample(frac=1,random_state=42)
data = data.reset_index(drop=True)

data

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,8856a9d8e2b4,0.130327,4748.74422,85.200147,6.460342,8.138688,3.255682,0.025578,7.968879,1.229900,...,4.215218,1.27426,0.067730,28.533885,72.611063,313.869492,24.971631,27.576984,0.271174,0
1,8d0d4a6510bf,0.337567,1250.66568,85.200147,14.097544,17.016780,4.624398,0.025578,17.922888,1.229900,...,0.173229,0.49706,0.067730,17.887746,72.611063,59497.505590,23.059037,101.825480,21.978000,0
2,0c3a256bcec7,0.132463,2160.54454,85.200147,20.300835,8.138688,4.810437,0.025578,9.371074,5.158552,...,0.173229,1.00514,0.419926,11.618198,72.611063,21451.810610,37.265815,51.308828,21.978000,0
3,add78517b003,0.329021,3828.91916,85.200147,28.566728,8.138688,6.422775,0.025578,13.007328,1.229900,...,6.973634,1.18378,0.345423,33.031402,72.611063,5892.167286,37.306746,117.352644,0.166770,0
4,a082f422799b,0.311929,2831.90336,85.200147,16.829098,8.138688,3.605613,0.025578,12.919100,6.803104,...,9.592530,2.01956,1.097226,23.720100,72.611063,16285.250560,34.629486,37.121676,0.075126,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,f405c0b5836f,0.508487,3692.13592,110.621718,13.859075,8.138688,5.377413,0.025578,11.608284,3.756466,...,14.201978,1.85339,0.284466,18.650510,75.393773,946.472364,45.835278,18.729304,0.043662,1
444,80841417ca53,0.542671,4857.60308,129.433158,21.849335,14.727774,6.644250,0.147378,8.671552,1.229900,...,10.805200,1.08286,0.663754,27.617638,72.611063,1896.371757,37.433260,59.163716,0.095221,0
445,af0802c15f01,0.273472,2341.58474,85.200147,17.392752,8.138688,2.852598,0.051156,7.322924,1.229900,...,0.173229,0.49706,0.115141,10.492656,72.611063,24323.704650,39.282597,30.275012,21.978000,0
446,f62ef7cc0a22,0.521306,4998.31277,85.200147,28.257028,8.138688,4.252320,0.025578,6.711630,1.229900,...,7.230382,1.55179,0.507975,20.603930,216.055347,2790.445788,39.178409,101.053880,0.198861,0


In [10]:
from sklearn.preprocessing import LabelEncoder

data['EJ'] = LabelEncoder().fit_transform(data['EJ'])
data=data.drop(['Id'], axis=1)

In [11]:
val['EJ'] = LabelEncoder().fit_transform(val['EJ'])
val=val.drop(['Id'], axis=1)

In [12]:
def competition_log_loss(y_pred, y_true):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    y_pred = y_pred.T[1]
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    w_0 = (N_0 + N_1) / N_0
    w_1 = (N_0 + N_1) / N_1
    # Calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # Calculate the average log loss for each class
    sum_0 = -np.sum((1 - y_true) * np.log(p_0))
    sum_1 = -np.sum(y_true * np.log(p_1))
    to_output = ((w_0 / N_0) * sum_0 + (w_1 / N_1) * sum_1) / (w_0 + w_1)

    return to_output

In [13]:
x=data.drop(["Class"], axis=1)
y=data["Class"]

In [14]:
import lightgbm as lgbm
from xgboost import XGBClassifier


In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)

In [16]:
lgb_train = lgbm.Dataset(x_train, y_train)
params={   
                                  'objective':'binary',
        
    'learning_rate': 0.19852198534809998,
'num_leaves': 900,
'max_depth': 4,
'lambda_l1': 0,
'lambda_l2': 5,
'bagging_fraction': 0.8,
'bagging_freq': 1,
'feature_fraction': 0.7}
lgbm_model=lgbm.LGBMClassifier(**params)
lgbm_model.n_estimators=10000
#lgbm_model
lgbm_model.fit(x_train,y_train)

[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, feature_fraction=0.7,
               lambda_l1=0, lambda_l2=5, learning_rate=0.19852198534809998,
               max_depth=4, n_estimators=10000, num_leaves=900,
               objective='binary')

In [17]:
params_2={
    'objective':'binary:logistic',

    'max_depth': 7,
    'learning_rate': 0.040214432727764,
    'subsample': 0.5983103516248403,
    'colsample_bytree': 0.978275320520211,
    'gamma': 0.9914944413877345
}
xgb_model_1 = XGBClassifier(**params_2)
xgb_model_1.n_estimators = 250
xgb_model_1.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.978275320520211, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.9914944413877345, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.040214432727764, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=250, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [18]:
params_3={'n_estimators': 900, 'max_depth': 3, 'learning_rate': 0.018221830558939608, 'subsample': 0.8626278030304476, 'colsample_bytree': 0.9025807208270448, 'gamma': 0.023647124186862576}
xgb_model_2 = XGBClassifier(**params_3)
xgb_model_2.n_estimators = 900
xgb_model_2.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9025807208270448, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.023647124186862576, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.018221830558939608, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=900, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [19]:
params_4={'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.03819123009105624, 'subsample': 0.6799023747645844, 'colsample_bytree': 0.7038174545826303, 'gamma': 0.15046415857565873}
xgb_model_3 = XGBClassifier(**params_4)
xgb_model_3.n_estimators = 500
xgb_model_3.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7038174545826303, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.15046415857565873, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.03819123009105624, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [20]:
params_5={'n_estimators': 950, 'max_depth': 4, 'learning_rate': 0.012156785140011915, 'subsample': 0.6030860268434596, 'colsample_bytree': 0.9288775819158103, 'gamma': 0.00272925544160879}
xgb_model_4 = XGBClassifier(**params_5)
xgb_model_4.n_estimators = 950
xgb_model_4.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9288775819158103, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.00272925544160879, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.012156785140011915, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=950, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [21]:
lgbm_pred = lgbm_model.predict_proba(x_test)
xgb_pred_1 = xgb_model_1.predict_proba(x_test)
xgb_pred_2 = xgb_model_2.predict_proba(x_test)
xgb_pred_3=xgb_model_3.predict_proba(x_test)
xgb_pred_4=xgb_model_4.predict_proba(x_test)

In [22]:
competition_log_loss(lgbm_pred,y_test)

0.4597618914623481

In [23]:
competition_log_loss(xgb_pred_1,y_test)

0.42208121121625053

In [24]:
competition_log_loss(xgb_pred_2,y_test)

0.3629922834563446

In [25]:
competition_log_loss(xgb_pred_3,y_test)

0.39661444960914266

In [26]:
competition_log_loss(xgb_pred_4,y_test)

0.40390593543379016

In [27]:
stack_pred = np.column_stack((lgbm_pred, xgb_pred_1,xgb_pred_2, xgb_pred_3,xgb_pred_4))


In [28]:
final_model = XGBClassifier()
final_model.fit(stack_pred, y_test)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [29]:
final_pred = final_model.predict_proba(stack_pred)
print("Log loss:", competition_log_loss( final_pred,y_test))

Log loss: 0.12321882279867906


In [30]:
lgbm_pred_val = lgbm_model.predict_proba(val)
xgb_pred_1_val = xgb_model_1.predict_proba(val)
xgb_pred_2_val = xgb_model_2.predict_proba(val)
xgb_pred_3_val= xgb_model_3.predict_proba(val)
xgb_pred_4_val= xgb_model_4.predict_proba(val)

# Stack predictions
stack_pred = np.column_stack((lgbm_pred_val, xgb_pred_1_val, xgb_pred_2_val, xgb_pred_3_val,xgb_pred_4_val))

# Make predictions on stacked predictions
new_pred = final_model.predict_proba(stack_pred)

In [31]:
new_pred=pd.DataFrame(new_pred)

In [32]:
new_pred

,0,1
0,0.526222,0.473778
1,0.526222,0.473778
2,0.526222,0.473778
3,0.526222,0.473778
4,0.526222,0.473778


In [33]:
sub

,Id,class_0,class_1
0,00eed32682bb,0.5,0.5
1,010ebe33f668,0.5,0.5
2,02fa521e1838,0.5,0.5
3,040e15f562a2,0.5,0.5
4,046e85c7cc7f,0.5,0.5


In [34]:
sub['class_0']=new_pred[0]

In [35]:
sub['class_1']=new_pred[1]

In [36]:
sub

,Id,class_0,class_1
0,00eed32682bb,0.526222,0.473778
1,010ebe33f668,0.526222,0.473778
2,02fa521e1838,0.526222,0.473778
3,040e15f562a2,0.526222,0.473778
4,046e85c7cc7f,0.526222,0.473778


In [37]:
sub.to_csv("submission.csv", index=False)